## Imports

In [109]:
# DO NOT CHANGE THESE LINES.
using Suppressor
@suppress begin
    using DataFrames, CSV, Random, Statistics, Serialization, LazyJSON, StatsBase, DecisionTree, ScientificTypes, MLJ
end


## Paths

In [110]:
# DO NOT CHANGE THESE LINES 
ROOT_DIR = dirname(pwd())
MODEL_INPUTS_OUTPUTS = joinpath(ROOT_DIR, "model_inputs_outputs")
INPUT_DIR = joinpath(MODEL_INPUTS_OUTPUTS, "inputs")
INPUT_SCHEMA_DIR = joinpath(INPUT_DIR, "schema")
DATA_DIR = joinpath(INPUT_DIR, "data")
OUTPUT_DIR = joinpath(MODEL_INPUTS_OUTPUTS, "outputs")
TRAIN_DIR = joinpath(DATA_DIR, "training")
TEST_DIR = joinpath(DATA_DIR, "testing")
MODEL_PATH = joinpath(MODEL_INPUTS_OUTPUTS, "model")
MODEL_ARTIFACTS_PATH = joinpath(MODEL_PATH, "artifacts")
OHE_ENCODER_FILE = joinpath(MODEL_ARTIFACTS_PATH, "ohe.jld2")
PREDICTOR_DIR_PATH = joinpath(MODEL_ARTIFACTS_PATH, "predictor")
PREDICTOR_FILE_PATH = joinpath(PREDICTOR_DIR_PATH, "predictor.ser")
IMPUTATION_FILE = joinpath(MODEL_ARTIFACTS_PATH, "imputation.ser")
TOP_CATEGORIES = joinpath(MODEL_ARTIFACTS_PATH, "top_categories.ser")
PREDICTIONS_DIR = joinpath(OUTPUT_DIR, "predictions")
PREDICTIONS_FILE = joinpath(PREDICTIONS_DIR, "predictions.csv")
TARGET_LEVELS = joinpath(MODEL_ARTIFACTS_PATH, "target_levels.ser")



if !isdir(MODEL_ARTIFACTS_PATH)
    mkdir(MODEL_ARTIFACTS_PATH)
end
if !isdir(PREDICTOR_DIR_PATH)
    mkdir(PREDICTOR_DIR_PATH)
end

### Reading the schema

In [111]:
# Reading a schema from a JSON file and extracting features
file_name = first(filter(x -> endswith(x, "json"), readdir(INPUT_SCHEMA_DIR)))
schema_path = joinpath(INPUT_SCHEMA_DIR, file_name)
schema_string = read(schema_path, String)  # Read file content as a string
schema = LazyJSON.parse(schema_string)
features = schema["features"]

# Identifying numeric, categorical, and nullable features
numeric_features = String[]
categorical_features = String[]
nullable_features = String[]

for f in features
    if f["dataType"] == "CATEGORICAL"
        push!(categorical_features, f["name"])
    else
        push!(numeric_features, f["name"])
    end
    if f["nullable"]
        push!(nullable_features, f["name"])
    end
end

# Extracting ID and target features
id_feature = schema["id"]["name"]
target_feature = schema["target"]["name"]
target_classes = schema["target"]["classes"]

3-element LazyJSON.Array{Nothing, String}:
 0
 1
 2

### Reading test data.

In [112]:
file_name = filter(x -> occursin(".csv", x), readdir(TEST_DIR))[1]
file_path = joinpath(TEST_DIR, file_name)
df = DataFrame(CSV.File(file_path))

,id,color,number
,String7,String7?,Float64?
1,WL7HCI,Blue,missing
2,3WY4QM,Red,45.0
3,Z3IWZH,Green,40.0
4,2K5IWJ,Green,12.0
5,YH1373,Green,80.0
6,AQGWOL,Green,47.0
7,MPPCVX,Blue,90.0
8,8UI6PP,missing,55.0
9,DJLE12,Red,74.0


## Data preprocessing
Note that when we work with testing data, we have to impute using the same values learned during training. This is to avoid data leakage.

In [113]:
imputation_values = open(deserialize, IMPUTATION_FILE)
for column in nullable_features
    df[!, Symbol(column)] .= coalesce.(df[!, Symbol(column)], get(imputation_values, string(column), missing))
end

# Saving the id column in a different variable
ids = df[!, Symbol(id_feature)]

# Dropping the id from the DataFrame
select!(df, Not([Symbol(id_feature)]))

,color,number
,String7,Float64
1,Blue,45.0
2,Red,45.0
3,Green,40.0
4,Green,12.0
5,Green,80.0
6,Green,47.0
7,Blue,90.0
8,Blue,55.0
9,Red,74.0


### Encoding
We encode the data using the same encoder that we saved during training.

In [114]:
loaded_top_categories = open(deserialize, TOP_CATEGORIES)

# Function to one-hot encode only the top 3 categories
function one_hot_top_categories!(df, top_categories)
    for (feature, top_cats) in top_categories
        if length(top_cats) == 2  # Handle the binary case
            # Assuming the first category in top_cats is treated as 'true'
            new_col_name = "$(feature)_binary"
            df[!, new_col_name] = df[!, feature] .== top_cats[1]
        else  # Handle the general case
            for cat in top_cats
                new_col_name = "$(feature)_$(cat)"
                df[!, new_col_name] = df[!, feature] .== cat
            end
        end
        select!(df, Not(Symbol(feature)))  # Drop the original feature column
    end
end



one_hot_top_categories!(df, loaded_top_categories)

### Making predictions & Creating Predictions DataFrame
Using the model saved during training.

In [115]:
model = open(deserialize, PREDICTOR_FILE_PATH)
predictions = apply_forest_proba(model, Matrix(df), [string(item) for item in target_classes])
predictions = DataFrame(predictions, Symbol.(target_classes))
insertcols!(predictions, 1, id_feature => ids)
CSV.write(PREDICTIONS_FILE, predictions)

LoadError: MethodError: no method matching String(::LazyJSON.Number{String})

[0mClosest candidates are:
[0m  String([91m::T[39m) where T<:InlineString
[0m[90m   @[39m [35mInlineStrings[39m [90m~/.julia/packages/InlineStrings/rlLZO/src/[39m[90m[4mInlineStrings.jl:133[24m[39m
[0m  String([91m::CategoricalArrays.CategoricalValue{<:AbstractString}[39m)
[0m[90m   @[39m [32mCategoricalArrays[39m [90m~/.julia/packages/CategoricalArrays/0yLZN/src/[39m[90m[4mvalue.jl:116[24m[39m
[0m  String([91m::WeakRefStrings.WeakRefString[39m)
[0m[90m   @[39m [36mWeakRefStrings[39m [90m~/.julia/packages/WeakRefStrings/31nkb/src/[39m[90m[4mWeakRefStrings.jl:82[24m[39m
[0m  ...
